In [1]:
import pandas as pd
import sqlite3
import joblib
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# 📦 Conexión a la base de datos
conn = sqlite3.connect("db/market_place.db")

# 📥 Cargar tabla de órdenes
df_orders = pd.read_sql_query("SELECT order_id, product_id FROM orders", conn)

# 🧹 FILTRO 1: Eliminar productos poco frecuentes
min_freq = 100  # puedes ajustar este valor
frecuentes = df_orders['product_id'].value_counts()
productos_validos = frecuentes[frecuentes >= min_freq].index
df_filtrado = df_orders[df_orders['product_id'].isin(productos_validos)]

# 🧹 FILTRO 2: Muestrear pedidos (si quieres limitar la escala)
n_pedidos = 500_000
pedidos_sample = df_filtrado['order_id'].drop_duplicates().sample(n=n_pedidos, random_state=42)
df_sampled = df_filtrado[df_filtrado['order_id'].isin(pedidos_sample)]

# 🧺 Agrupar productos por pedido
basket = df_sampled.groupby('order_id')['product_id'].apply(list).reset_index()

# 🧠 Transformar a formato booleano para Apriori
te = TransactionEncoder()
te_ary = te.fit(basket['product_id']).transform(basket['product_id'])

df_basket = pd.DataFrame(te_ary, columns=te.columns_)


In [6]:
# ⚙️ Algoritmo Apriori
frequent_itemsets = apriori(df_basket, min_support=0.01, use_colnames=True)

# 📏 Generar reglas de asociación
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

In [7]:
# 💾 Guardar reglas entrenadas
joblib.dump(rules, "models/MBA/MBA_reglas_apriori.joblib")
print(f"✅ Reglas generadas: {len(rules)} y guardadas con éxito.")

rules.to_csv("models/MBA/rules.csv",index=False)

✅ Reglas generadas: 28 y guardadas con éxito.
